In [1]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
from google.colab import drive
drive.mount('/content/drive')




  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=727ee5f19ba4129fdaee8988782842017484169992e57073731ea8829e057ffa
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ngrok authtoken 2TTqM4LyLrUj5QAUrZBd8ahxSA9_561SSmCMHhHTzwhHTiWbR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
!pip install pandas openpyxl


In [4]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [5]:
import pandas as pd


In [6]:
from datetime import datetime
import pytz
import time

In [ ]:
#https://www.youtube.com/watch?v=vieoHqt7pxo
#servidor

import numpy as np
import pandas as pd
import tensorflow as tf
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import joblib
from tensorflow.keras.models import load_model

app = Flask(__name__)
run_with_ngrok(app)
reshaped_motion_data = []

def reshape_data(json_data):


    motion_data_test = pd.DataFrame(json_data, columns=['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ','Humidity'])

    motion_data_array = motion_data_test

    reshaped_motion_data.append(motion_data_array.iloc[0].values.tolist())
    #print(reshaped_motion_data)
    return reshaped_motion_data


def danger_level(data):

  model = load_model('/content/drive/My Drive/hackamodel.h5')
  single_instance_prediction = model.predict(data)
  single_instance_prediction = single_instance_prediction[-1][-1][-1]
  if(single_instance_prediction < 0.5 and data[-1][-1][-1] < 0.5):

    #print('Danger Level 3')
    return("Danger Level 3")
  elif(single_instance_prediction < 0.5 and data[-1][-1][-1] > 0.5):
    #print('Danger Level 2')
    return("Danger Level 2")
  elif(single_instance_prediction > 0.5 and data[-1][-1][-1] < 0.5):
   # print('Danger Level 1')
    return("Danger Level 1")
  elif(single_instance_prediction > 0.5 and data[-1][-1][-1] > 0.5):
    #print('Danger Level 4')
    return("Danger Level 4")

@app.route('/data_with_humidity', methods=['POST'])
def predict_data():
    prediction = "";
    json_data = request.get_json()
    reshaped_data = reshape_data(json_data)
    print(len(reshaped_data))
    if(len(reshaped_data)<20):
     print("vou esperar")
    elif(len(reshaped_data)==20):
      reshaped_data = np.array(reshaped_data)
      import pickle

      with open('/content/drive/My Drive/minmax_scaler.pkl', 'rb') as file:
        loaded_scaler = pickle.load(file)

      reshaped_data = loaded_scaler.transform(reshaped_data)
      reshaped_data = reshaped_data.reshape(1,reshaped_data.shape[0], reshaped_data.shape[1])
      predict_list =  danger_level(reshaped_data)
      prediction = predict_list
      print(prediction)

    else:
      reshaped_data = np.array(reshaped_data)
      import pickle

      with open('/content/drive/My Drive/minmax_scaler.pkl', 'rb') as file:
        loaded_scaler = pickle.load(file)

      reshaped_data = loaded_scaler.transform(reshaped_data)
      reshaped_data = reshaped_data.reshape(1,reshaped_data.shape[0], reshaped_data.shape[1])
      predict_list =  danger_level(reshaped_data[:,-20:,:])
      prediction = predict_list
      print(prediction)
    return jsonify({"Predicted Class": prediction})


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d779-35-247-101-154.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
